In [23]:
import psycopg2
import pandas as pd
from datetime import datetime, timedelta

# Function to get the start and end dates of the fiscal quarter
def get_fiscal_quarter_dates(date):
    # Convert the input datetime to a pandas Timestamp
    date = pd.Timestamp(date)
    # Get the quarter number
    quarter = date.quarter
    # Calculate the start date of the quarter
    quarter_start_date = pd.Timestamp(year=date.year, month=3 * (quarter - 1) + 1, day=1)
    # Calculate the end date of the quarter by getting the first day of the next quarter and subtracting 1 day
    quarter_end_date = (quarter_start_date + pd.offsets.QuarterEnd()).date()  # pandas automatically moves to the end of the quarter, so just add a quarter end offset
    # Format dates as strings
    return quarter_start_date.strftime('%Y-%m-%d'), quarter_end_date.strftime('%Y-%m-%d')

# Database connection parameters
params = {
    'host': 'public.xbrl.us',
    'port': '5432',
    'database': 'edgar_db',
    'user': 'xuspwr250',
    'password': 'lwiAJZhJQhssXI#'
}

# Get the date from the user or any source
input_date = datetime(2021, 1, 1)

sic_code = '357'  # Modify this as needed

# Get the start and end dates of the fiscal quarter
start_date, end_date = get_fiscal_quarter_dates(input_date)

sql_query = f"""
SELECT report.report_id, report.entity_name, report.reporting_period_end_date, report.properties->>'document_type' AS document_type
FROM report
WHERE report.reporting_period_end_date >= '{start_date}'
AND report.reporting_period_end_date <= '{end_date}'
AND (report.properties->>'standard_industrial_classification')::text LIKE '{sic_code}%'
AND report.source_id = (SELECT source_id FROM source WHERE source_name = 'SEC')
AND report.properties->>'document_type' IN ('10-Q', '10-K');
"""

# Connect to the database
try:
    conn = psycopg2.connect(**params)
    cursor = conn.cursor()
    cursor.execute(sql_query)
    # Fetch all rows
    rows = cursor.fetchall()
    # Get column names
    col_names = [desc[0] for desc in cursor.description]
    # Create DataFrame
    df = pd.DataFrame(rows, columns=col_names)
    # Close cursor and connection
    cursor.close()
    conn.close()
except (Exception, psycopg2.DatabaseError) as error:
    print(error)

# Display the DataFrame
df

,report_id,entity_name,reporting_period_end_date,document_type
0,337621,"NetApp, Inc.",2021-01-29,10-Q
1,355356,INTRUSION INC,2021-03-31,10-Q
2,350808,"OMNICELL, INC",2021-03-31,10-Q
3,359781,QUANTUM CORP /DE/,2021-03-31,10-K
4,349959,"JUNIPER NETWORKS, INC.",2021-03-31,10-Q
5,349230,"EXTREME NETWORKS, INC.",2021-03-31,10-Q
6,355633,LOGITECH INTERNATIONAL S.A.,2021-03-31,10-K
7,359013,NewHold Investment Corp.,2021-03-31,10-Q
8,352757,INTERLINK ELECTRONICS INC,2021-03-31,10-Q
9,353199,"MITEK SYSTEMS, INC.",2021-03-31,10-Q
